In [27]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

from sklearn import metrics
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin, clone
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.externals import joblib

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [28]:
target = '标签'
uid = '申请编号'

def get_time():
    now = datetime.datetime.now().strftime("%m-%d %H:%M")
    print(now)

def calc_auc(y_test, y_proba):
    auc = round(metrics.roc_auc_score(y_test, y_proba), 3)
    return auc

def ks_score(y_test, y_proba):
    scale = 4
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba, pos_label=1)
    KS = round(max(list(tpr-fpr)), scale)
    return KS

In [3]:
############## Load

# X = pd.read_csv('./tmp/1_X.csv', header=0, index_col=0)
# X.shape
# X.head()

# y = pd.read_csv('./tmp/1_y.csv', header=0, index_col=0)
# y.shape
# y.head()

X = pd.read_csv('./tmp/train_d12_d3_dum.csv', header=0, index_col=0).drop(uid, axis=1).values
# X = pd.read_csv('./tmp/1_X.csv', header=0, index_col=0).values
len(X)
len(X[0])

y = pd.read_csv('./tmp/1_y.csv', header=0, index_col=0)['0'].values
len(y)

140000

390

140000

In [31]:
########## Grid Search

scale_pos_weight = 119/21

param_general = {
#     'n_iter' : 100,
#     'cv' : 5, 
    'n_iter' : 50,
    'cv' : 5, 
    'scoring' : 'roc_auc', 
    'n_jobs' : -1, 
    'random_state' : 123, 
    'verbose' : 1}

# RF
param_dist_rf = {
    # Shape
    'n_estimators' : range(50, 500, 50),
#     'n_estimators' : range(5, 10),
    'max_depth' : range(3, 10),
    'min_samples_split' : range(50, 100, 10),
    'min_samples_leaf' : range(50, 100, 10),
    # Sample
    'class_weight' : ['balanced', None],
    'max_features' : ['sqrt', 'log2'],
    # Objective
    'criterion' : ['gini', 'entropy']
}

# XGB
param_dist_xgb = {
    # Shape
    'n_estimatores' : range(50, 500, 50),
#     'n_estimators' : range(5, 10),
    'max_depth' : range(3, 10),
    'min_child_weight' : range(1, 9, 1), # 最小叶子节点样本权重和
    # Sample
    'scale_pos_weight' : [scale_pos_weight, 1],
    'subsample' : np.linspace(0.5, 0.9, 5),
    'colsample_bytree' : np.linspace(0.5, 0.9, 5),
    'colsample_bylevel' : np.linspace(0.5, 0.9, 5),
    # Algo
    'eta' : np.linspace(0.01, 0.2, 20), # Learning_rate
    'alpha' : np.linspace(0, 1, 10),
    'lambda' : range(0, 50, 5),
    'early_stopping_rounds' : range(10, 20, 5)
}

# LGB
param_dist_lgb = {
    # Shape
    'num_boost_round' : range(50, 500, 50),
#     'num_boost_round' : range(50, 100, 10),
    'num_leaves' : range(2**3, 2**10, 100),
    'min_data_in_leaf' : range(50, 100, 10),
    'min_child_weight' : range(1, 9, 1), # 最小叶子节点样本权重和
    # Sample
    'is_unbalance' : [True, False],
    'bagging_freq': range(2, 10), # >0 enable bagging_fraction
    'bagging_fraction': np.linspace(0.5, 0.9, 5),
    'feature_fraction': np.linspace(0.5, 0.9, 5),
    'subsample' : np.linspace(0.5, 0.9, 5),
    # Algo
    'learning_rate':np.linspace(0.01, 0.2, 20),    
    'lambda_l1': np.linspace(0, 1, 10),
    'lambda_l2': range(0, 50, 5),
    'cat_smooth': range(1, 40, 5)
#     'early_stopping_rounds' : range(10, 20, 5)
}

##########

# RF
param_fixed_rf = {
    'n_jobs' : -1,
    'oob_score' : True,
    'random_state':123,
    'verbose':0
}

# XGB
param_fixed_xgb = {
    'n_jobs' : -1,
    'eval_metric': 'auc',
    'seed' : 123,
    'silent' : 1,
    'verbose_eval':0
}

# LGB
param_fixed_lgb = {
    'n_jobs' : -1,
    'metric' : 'auc',
    'bagging_seed':123,
    'feature_fraction_seed':123,
    'verbose_eval' : 0
}

In [ ]:
############## RF

get_time()
''' Baseline '''
baseline = RandomForestClassifier(**param_fixed_rf)
baseline.fit(X, y)
pred_baseline = baseline.predict_proba(X)
ks_score(y, pred_baseline[:,1])

get_time()
''' Best '''
grid = RandomizedSearchCV(RandomForestClassifier(**param_fixed_rf), param_dist_rf, **param_general)
grid.fit(X, y)
grid.best_score_
best_params = grid.best_params_
np.save('./model/base_rf.npy', best_params)
get_time()

# ''' Test Clone Model '''
# model1 = grid.best_estimator_
# model1.fit(X, y)
# ks_score(y, model1.predict_proba(X)[:,1])
# 
# ''' Test Save Params '''
# best_params_load = np.load('./model/base_rf.npy', allow_pickle=True).item()
# model2_params = {**best_params_load, **param_fixed_rf}
# model2 = RandomForestClassifier(**model2_params)
# model2.fit(X, y)
# ks_score(y, model2.predict_proba(X)[:,1])

10-20 09:51


' Baseline '

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=True, random_state=123, verbose=0,
                       warm_start=False)

0.9852

10-20 09:51


' Best '

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
############## XGB

get_time()
''' Baseline '''
baseline = XGBClassifier(**param_fixed_xgb)
baseline.fit(X, y)
pred_baseline = baseline.predict_proba(X)
ks_score(y, pred_baseline[:,1])

get_time()
''' Best '''
grid = RandomizedSearchCV(XGBClassifier(**param_fixed_xgb), param_dist_xgb, **param_general)
grid.fit(X, y)
grid.best_score_
best_params = grid.best_params_
np.save('./model/base_xgb.npy', best_params)
get_time()
get_time()

# ''' Test Clone Model '''
# model1 = grid.best_estimator_
# model1.fit(X, y)
# ks_score(y, model1.predict_proba(X)[:,1])

# ''' Test Save Params '''
# best_params_load = np.load('./model/base_xgb.npy', allow_pickle=True).item()
# model2_params = {**best_params_load, **param_fixed_xgb}
# model2 = XGBClassifier(**model2_params)
# model2.fit(X, y)
# ks_score(y, model2.predict_proba(X)[:,1])

In [ ]:
############## LGB

get_time()
''' Baseline '''
baseline = LGBMClassifier(**param_fixed_lgb)
baseline.fit(X, y)
pred_baseline = baseline.predict_proba(X) #, num_iteration=baseline.best_iteration_)
ks_score(y, pred_baseline[:,1])

get_time()
''' Best '''
grid = RandomizedSearchCV(LGBMClassifier(**param_fixed_lgb), param_dist_lgb, **param_general)
grid.fit(X, y)
grid.best_score_
best_params = grid.best_params_
np.save('./model/base_lgb.npy', best_params)
get_time()

# ''' Test Clone Model '''
# model1 = grid.best_estimator_
# model1.fit(X, y)
# ks_score(y, model1.predict_proba(X)[:,1])

# ''' Test Save Params '''
# best_params_load = np.load('./model/base_lgb.npy', allow_pickle=True).item()
# model2_params = {**best_params_load, **param_fixed_lgb}
# model2 = LGBMClassifier(**model2_params)
# model2.fit(X, y)
# ks_score(y, model2.predict_proba(X)[:,1])